In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

ModuleNotFoundError: No module named 'google'

In [4]:
import os
import sys
dir_path = '/content/gdrive/My Drive/urbcomp_fall2025/assignment_1'
sys.path.append(dir_path)

In [ ]:
!python --version

In [ ]:
# including modules
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import seaborn as sns
import operator
import numpy as np
import geopandas as gp
%load_ext autoreload
%autoreload 2
from utils import airtraffic_helpers
import networkx as nx
import community
import random
from shapely.geometry import Point

In [ ]:
# Dometics airport details
airports_df = pd.read_csv("{}/dataset/288804893_T_MASTER_CORD.csv".format(dir_path))
print('Shape of the dataframe:',airports_df.shape,'\n')
print('Printing one record:',airports_df[:1].T)

In [ ]:
# Dometics airlines connections
trips_df = pd.read_csv("{}/dataset/288798530_T_T100D_MARKET_ALL_CARRIER.csv".format(dir_path))
print('Shape of the dataframe:',trips_df.shape,'\n')
print('Printing one record:',trips_df[:1].T)

In [ ]:
# Extracting edges - we consider a connection from one airport to another as an edge.
# Note: these edges will be directed.
edges = list(zip(trips_df['ORIGIN_AIRPORT_ID'],trips_df['DEST_AIRPORT_ID']))

In [2]:
# creating directed and undirected graphs based on airports and their connections
G = nx.DiGraph()
G.add_edges_from(edges)
G_undirected = nx.Graph()
G_undirected.add_edges_from(edges)

NameError: name 'nx' is not defined

In [ ]:
# printing the total number of nodes and edges of directed graph
print('Total number of airports:',len(list(G.nodes)))
print('Total number of connections:',len(list(G.edges)))

In [ ]:
# creating a GeoDataframe for plotting airports on a map
airport_ids = list(G.nodes)
edgelist = list(G.edges)
geo_stations = airtraffic_helpers.get_geodataframe_airports(airports_df,airport_ids)

In [ ]:
# reading the shape file of US
shp_us = gp.GeoDataFrame.from_file("{}/dataset/Igismap/Alabama_AL4_US_Poly.shp".format(dir_path))
shp_us=shp_us.to_crs({'init':'epsg:4326'})
shp_us.plot(figsize=(100,100),color='g',alpha=0.75)

In [ ]:
# plotting the nodes in the shape file of US
plt.style.use("default")
%matplotlib inline
fig, ax = plt.subplots(1,1,figsize=(50,50))
base = shp_us.plot(ax=ax, color='gray', alpha=0.2)
geo_stations.plot(ax=base, marker="o", color="r", markersize=10,alpha=0.8, zorder=0)
_ = ax.axis('off')
ax.set_title("Plot of airports in United States",fontsize=20)
fig.tight_layout()
fig.savefig("{}/figures/us_airports_nodes.pdf".format(dir_path))

In [ ]:
# displaying the network based on actual geographical placement of nodes
position_dict = {} # will contain the node coordinates
for item in list(geo_stations.station_ids):
  position_dict[item] = geo_stations[geo_stations.station_ids==item].geometry.values[0].x,geo_stations[geo_stations.station_ids==item].geometry.values[0].y
fig, ax = plt.subplots(1,1,figsize=(100,40))
nx.draw(G,pos=position_dict)
fig.tight_layout()
fig.savefig("{}/figures/us_airports_network_withposition.pdf".format(dir_path))

In [ ]:
# displaying the network with optimal placement of nodes
airtraffic_helpers.plot_network(G,title="",edgealpha=0.08,node_dist=1,nodesize=20,savefig=False,figsize=(15,15))